## Graph pre-process
Please note, this is created specifically for Vienna, other cities may have to be processed differently in regard to elevation data.



In [1]:
#Setup:
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt
from itertools import islice
import numpy as np
import pandas as pd

In [2]:
city = "Vienna, Austria" # making this a variable, since it needed for the features module 

In [8]:
G = ox.graph.graph_from_place(city, network_type="walk") # getting the graph of walking paths in Vienna 
#Creating a subgraph of nodes that have the streetcount as 2 or higher

selected_nodes = [n for n, attr in G.nodes(data=True) if attr.get('street_count', 0) > 1]

SG = G.subgraph(selected_nodes)

TypeError: Nominatim did not geocode query 'Vienna, Austria' to a geometry of type (Multi)Polygon.

In [27]:
# Projecting the graph so it alligns with the StadtWien elevation data
SG_proj = ox.projection.project_graph(SG, to_crs = 31256)

In [28]:
# Adding paths to our elevation files
raster_path_names = ['15_2_dgm.tif', '24_4_dgm.tif', '45_3_dgm.tif', '56_1_dgm.tif', '34_2_dgm.tif', '38_1_dgm.tif', '47_4_dgm.tif', '26_3_dgm.tif', '53_4_dgm.tif', '48_3_dgm.tif', '57_2_dgm.tif', '35_1_dgm.tif', '43_2_dgm.tif', '26_2_dgm.tif', '32_2_dgm.tif', '44_1_dgm.tif', '17_4_dgm.tif', '36_4_dgm.tif', '22_4_dgm.tif', '43_3_dgm.tif', '27_1_dgm.tif', '15_3_dgm.tif', '45_2_dgm.tif', '33_1_dgm.tif', '34_3_dgm.tif', '55_4_dgm.tif', '25_4_dgm.tif', '44_3_dgm.tif', '35_2_dgm.tif', '43_1_dgm.tif', '57_1_dgm.tif', '33_3_dgm.tif', '46_4_dgm.tif', '27_3_dgm.tif', '42_2_dgm.tif', '34_1_dgm.tif', '56_2_dgm.tif', '45_1_dgm.tif', '16_4_dgm.tif', '27_2_dgm.tif', '23_4_dgm.tif', '38_3_dgm.tif', '56_3_dgm.tif', '37_4_dgm.tif', '44_2_dgm.tif', '48_1_dgm.tif', '26_1_dgm.tif', '35_3_dgm.tif', '54_4_dgm.tif', '58_2_dgm.tif', '54_1_dgm.tif', '36_2_dgm.tif', '32_4_dgm.tif', '53_3_dgm.tif', '48_4_dgm.tif', '47_3_dgm.tif', '26_4_dgm.tif', '55_2_dgm.tif', '37_1_dgm.tif', '16_1_dgm.tif', '24_3_dgm.tif', '45_4_dgm.tif', '34_4_dgm.tif', '55_3_dgm.tif', '24_2_dgm.tif', '15_4_dgm.tif', '46_1_dgm.tif', '43_4_dgm.tif', '36_3_dgm.tif', '25_1_dgm.tif', '17_3_dgm.tif', '53_2_dgm.tif', '47_2_dgm.tif', '37_2_dgm.tif', '55_1_dgm.tif', '46_3_dgm.tif', '27_4_dgm.tif', '33_4_dgm.tif', '16_2_dgm.tif', '28_3_dgm.tif', '36_1_dgm.tif', '54_2_dgm.tif', '58_1_dgm.tif', '25_3_dgm.tif', '44_4_dgm.tif', '35_4_dgm.tif', '54_3_dgm.tif', '47_1_dgm.tif', '25_2_dgm.tif', '53_1_dgm.tif', '56_4_dgm.tif', '37_3_dgm.tif', '23_3_dgm.tif', '42_4_dgm.tif', '46_2_dgm.tif', '24_1_dgm.tif', '16_3_dgm.tif', '33_2_dgm.tif']
raster_paths = ["/Users/sofiiashome/Documents/Studying at WU/Bachelor's Thesis/Bachelor Thesis Coding/LoopBuddy/graph_preprocessing/Elevation tifs/"+x for x in raster_path_names]
print(raster_paths[0])

/Users/sofiiashome/Documents/Studying at WU/Bachelor's Thesis/Elevation tifs2/15_2_dgm.tif


In [29]:
SG_proj = ox.elevation.add_node_elevations_raster(SG_proj, raster_paths, cpus = 1) #Adds elevations from local raster files

#### Let's add more node labels.

Here we add: stoplights, pavement type, stairs

Relevant tag links:

Stoplights: https://wiki.openstreetmap.org/wiki/Tag:highway%3Dtraffic_signals

Pavement type: https://wiki.openstreetmap.org/wiki/Key:surface

Stairs :  https://wiki.openstreetmap.org/wiki/Tag:highway%3Dsteps

In [32]:
SG_unproj = ox.projection.project_graph(SG_proj, to_crs = 4326) # Projecting back to EPSD: 4326, as this is the CRS that OSMnx and OSM (data) # https://wiki.openstreetmap.org/wiki/Converting_to_WGS84

In [33]:
# Let's get the stoplight data
tags = {"highway":  "steps", "surface": True }#"surface": True ,"highway":"steps"

loaded_features = ox.features.features_from_place(city, tags) # getting the stoplight data from OSM 
# https://wiki.openstreetmap.org/wiki/Tag:highway%3Dtraffic_signals

# print(loaded_features)

In [34]:
# attaching it to the nearest nodes as attributes # code sources from "16-download-osm-geospatial-features.ipynb" in the OSMnx examples gallery
feature_points = loaded_features.representative_point()
nearest_nodes = ox.distance.nearest_nodes(SG_unproj, feature_points.x, feature_points.y)

print(nearest_nodes)

[     381234      392321     1438521 ...    25277154 11565229497
  6020323414]


In [35]:
# Setting stoplights,stairs to False for all nodes for consistency reasons
# Create a dict: {node_id: False for all nodes}
false_dict = {node: False for node in SG_unproj.nodes}

# Batch update
nx.set_node_attributes(SG_unproj, false_dict, "stoplights")
nx.set_node_attributes(SG_unproj, false_dict, "steps")

In [36]:
# in order to get the surfaces to paved vs not paved, we need to group the tags into what we considered paved vs not paved:
paved = [
    'paved', 'asphalt', 'chipseal', 'concrete', 'concrete:lanes', 'concrete:plates',
    'paving_stones', 'paving_stones:lanes', 'grass_paver', 'sett', 'unhewn_cobblestone',
    'cobblestone', 'cobblestone:flattened', 'bricks', 'metal', 'metal_grid', 'wood',
    'rubber', 'tiles', 'fibre_reinforced_polymer_grate', 'artificial_turf', 'acrylic',
    'carpet', 'plastic', 'tartan','stepping_stones'
]

unpaved = [
    'unpaved', 'compacted', 'fine_gravel', 'gravel', 'shells', 'rock', 'pebblestone',
    'ground', 'dirt', 'earth', 'grass', 'mud', 'sand', 'woodchips', 'snow', 'ice',
    'salt', 'clay'
]

In [37]:
# Reasoning why I am doing this, and not just using the already present highway tag, is because this tags the stoplights more correctly, and on each node that they are present on

# in these and the following functions we use get is because it avoid issues if the attribute is empty

leftover_null_nodes = []

useful_tags = ["steps", "surface"]
for node, feature in zip(nearest_nodes, loaded_features[useful_tags].to_dict(orient="records")): # here we only update the nodes that have stoplights (that's why we had to add the False values beforehand)    
    #Adding pavement info:
    if feature.get("surface") in paved:
        surface_state = "paved"
    elif feature.get("surface") in unpaved:
        surface_state = "unpaved"
    else: # handling null values by getting the vaue from adjacent nodes

        neighbor_surfaces = []
        for neighbor in SG_unproj.adj[node]: # checks neighbor surfaces
            neighbor_surface = SG_unproj.nodes[neighbor].get("surface")
            if neighbor_surface in paved:
                neighbor_surfaces.append("paved")
            elif neighbor_surface in unpaved:
                neighbor_surfaces.append("unpaved")
        
        if neighbor_surfaces:
            # Majority vote: pick most common
            surface_state = max(set(neighbor_surfaces), key=neighbor_surfaces.count)
        else:
            # Fallback if no neighbors have surface info:
            surface_state = "unknown"
            leftover_null_nodes.append(node)


    SG_unproj.nodes[node].update({"surface":surface_state})

    # Adding steps info:
    is_step = feature.get("steps") == "yes"
    SG_unproj.nodes[node].update({"steps": is_step})

In [38]:
iterations = 0
max_iterations = 1000  # safety net

while len(leftover_null_nodes) > 0 and iterations <= max_iterations: # loops over all the nodes again and again until "theoretically" there are no unlabelled nodes. Uses the logic that adjacent nodes have similar paving. 

    print(f"Iteration {iterations}, remaining nodes: {len(leftover_null_nodes)}")
    nodes_updated_this_pass = [] # to ensure at least some nodes got filled


    for node in leftover_null_nodes: 
        neighbor_surfaces = []
        for neighbor in SG_unproj.adj[node]: # checks neighbor surfaces
            neighbor_surface = SG_unproj.nodes[neighbor].get("surface")
            #print(f"Neighbor {neighbor} surface: {neighbor_surface}")
            #print(neighbor_surface)
            if neighbor_surface in paved:
                neighbor_surfaces.append("paved")
            elif neighbor_surface in unpaved:
                neighbor_surfaces.append("unpaved")
        #print(neighbor_surfaces)

        if neighbor_surfaces:
        # picking most common nodes
            surface_state = max(set(neighbor_surfaces), key=neighbor_surfaces.count)
            SG_unproj.nodes[node].update({"surface": surface_state})
            nodes_updated_this_pass.append(node)
        else:
        #if no neighbors have surface info
            surface_state = "unknown"  

    for node in nodes_updated_this_pass:
        leftover_null_nodes.remove(node)

    if not nodes_updated_this_pass: # to avoid the situation where isolated clusters keep the loop running
                print("No more updates possible, stopping recursion.")
                break
    
    iterations += 1

Iteration 0, remaining nodes: 1673
Iteration 1, remaining nodes: 1109
Iteration 2, remaining nodes: 1073
Iteration 3, remaining nodes: 1062
Iteration 4, remaining nodes: 1060
Iteration 5, remaining nodes: 1058
Iteration 6, remaining nodes: 1056
No more updates possible, stopping recursion.


In [39]:
# For some reason the stoplights break if you load them with everything, so here we go:

stoplights = ox.features.features_from_place(city, {"highway": "traffic_signals"}) # getting the stoplight data from OSM 
stoplights_points = stoplights.representative_point()
nn_stoplights = ox.distance.nearest_nodes(SG_unproj, stoplights_points.x, stoplights_points.y)

for node, feature in zip(nn_stoplights, stoplights[["highway"]].to_dict(orient="records")):
    is_stoplight = feature.get("highway") == "traffic_signals"
    SG_unproj.nodes[node].update({"stoplights": is_stoplight})


----
### Adding edge values for parametres: stoplights, pavements, steps
Basically the logic goes, that if if the edge's nodes have these things in one or other state, the edge should inherit that.

----

In [41]:
for node0, node1, key in SG_unproj.edges(keys=True):
    # Stoplights
    if (SG_unproj.nodes[node0].get("stoplights") == True) or (SG_unproj.nodes[node1].get("stoplights") == True):
        SG_unproj.edges[node0, node1, key]["stoplights"] = True
    else:
        SG_unproj.edges[node0, node1, key]["stoplights"] = False

    # Steps
    if (SG_unproj.nodes[node0].get("steps") == True) or (SG_unproj.nodes[node1].get("steps") == True):
        SG_unproj.edges[node0, node1, key]["steps"] = True
    else:
        SG_unproj.edges[node0, node1, key]["steps"] = False

    # Pavement:
    # now here, we have to think actually: the adjacent to the edge nodes might have different statuses

    # logic: 
        # if both the same -> that label
        # if diff or unknown -> unknown
            # because then we can apply a partial punishment to the unknown nodes, since in reality we don't know how much of the edge is which pavement type
    if SG_unproj.nodes[node0].get("surface") == SG_unproj.nodes[node1].get("surface"):
        SG_unproj.edges[node0, node1, key]["surface"] = SG_unproj.nodes[node0].get("surface")
    else:
        SG_unproj.edges[node0, node1, key]["surface"] = "unknown"


In [45]:
# impute missing edge speeds and calculate edge travel times with the speed module
SG_unproj = ox.routing.add_edge_speeds(SG_unproj)
SG_unproj = ox.routing.add_edge_travel_times(SG_unproj)

In [46]:
#Testing pickl
import pickle
graph_path = "/Users/sofiiashome/Documents/Studying at WU/Bachelor's Thesis/GraphSaves/Weighted1_9.pkl"
with open(graph_path, "wb") as f:
    pickle.dump(SG_unproj, f)

## Code used to process the original data into Float32 files, however the process may have to be toggled to your specific file format:
Original files with elevation data can be downloaded from: https://www.wien.gv.at/ma41datenviewer/public/
They were not included due to size

In [ ]:
# import rasterio
# import os


# raster_paths = [""] # Directory with the files to be converted

# # Output directory for converted files:
# output_dir = ""
# os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists

# # Process each file
# for tif_file in raster_paths:
#     try:
#         # Construct output filename:
#         file_name = os.path.basename(tif_file)
#         output_file = os.path.join(output_dir, file_name)

#         # Convert raster to Float32:
#         with rasterio.open(tif_file) as src:
#             data = src.read(1).astype("float32")  # Convert to Float32
#             profile = src.profile
#             profile.update(dtype="float32")

#             with rasterio.open(output_file, "w", **profile) as dst:
#                 dst.write(data, 1)

#         print(f"Converted: {tif_file} -> {output_file}")

#     except Exception as e:
#         print(f"Error processing {tif_file}: {e}")

# print("Conversion completed!")

In [ ]:
# import os
# 
# raster_paths = [""]
# if os.path.exists(raster_paths[0]):
#     print("File exists!")
# else:
#     print("File NOT found! Check the path.")